In [11]:
import pyowm
from pyowm.agroapi10.enums import SatelliteEnum, PresetEnum

owm = pyowm.OWM('de95041357fb3e1d49e788dc391d13a6')
mgr = owm.agro_manager()

In [12]:
list_of_polygons = mgr.get_polygons()
list_of_polygons 
# Poligon ID Bologna ==> 654a67516352a3c72c2cc042

UnauthorizedError: Invalid API Key provided

In [ ]:
#mgr.delete_polygon(list_of_polygons[0])

In [ ]:
# Example of how to take satellite imagery from a polygon
""" pol_id = '654a67516352a3c72c2cc042'  # your polygon's ID
acq_from = 1500336000 + 189216000  # 18 July 2023 --> 1689552000
acq_to = 1508976000 + 189216000    # 26 October 2023 --> 1698192000

# the most basic search ever: search all available images upon the polygon in the specified time frame 
results = mgr.search_satellite_imagery(pol_id, acq_from, acq_to, acquired_by=SatelliteEnum.SENTINEL_2.symbol,
                                       preset=PresetEnum.NDVI) """

" pol_id = '654a67516352a3c72c2cc042'  # your polygon's ID\nacq_from = 1500336000 + 189216000  # 18 July 2023 --> 1689552000\nacq_to = 1508976000 + 189216000    # 26 October 2023 --> 1698192000\n\n# the most basic search ever: search all available images upon the polygon in the specified time frame \nresults = mgr.search_satellite_imagery(pol_id, acq_from, acq_to, acquired_by=SatelliteEnum.SENTINEL_2.symbol,\n                                       preset=PresetEnum.NDVI) "

In [ ]:
# http://api.agromonitoring.com/agro/1.0/ndvi/history?start=1689552000&end=1698192000&polyid=654a67516352a3c72c2cc042&appid=221aefae29a0184792ac527d1d7f80fa
import requests
import json

# Define the API endpoint
url = "http://api.agromonitoring.com/agro/1.0/ndvi/history"

# Define the query parameters
params = {
    "start": "1689552000",  # Start time (Unix timestamp)
    "end": "1698192000",    # End time (Unix timestamp)
    "polyid": "654a67516352a3c72c2cc042",  # ID of the polygon
    "appid": "de95041357fb3e1d49e788dc391d13a6"  # Your API key
}

# Send a GET request to the API
response = requests.get(url, params=params)

# Parse the response as JSON
data = response.json()

# Print the data
print(json.dumps(data, indent=4))

{
    "title": "There is a problem upon receipt of the polygon",
    "message": "Not Found"
}


We now want to create a new polygon for each circoscrizione of the city. 

Documentation: https://agromonitoring.com/api/history-ndvi

In [ ]:
import json
from pyowm.utils.geo import Polygon as GeoPolygon
import requests
import json

NDVI_circoscrizioni = {}

with open('../data/geojson/circoscrizioni.geojson') as f:
    circoscrizioni = json.load(f)

url = "http://api.agromonitoring.com/agro/1.0/ndvi/history"

for feature in circoscrizioni['features']:
    current_coords = feature['geometry']['coordinates']
    gp = GeoPolygon(current_coords)

    the_new_polygon = mgr.create_polygon(gp, 'new_polygon')

    params = {
        "start": "1689552000",  # Start time (Unix timestamp)
        "end": "1698192000",    # End time (Unix timestamp)
        "polyid": the_new_polygon.id,  # ID of the new polygon
        "appid": "de95041357fb3e1d49e788dc391d13a6"  # Your API key
    }

    # Send a GET request to the API
    response = requests.get(url, params=params)

    # Parse the response as JSON
    data = response.json()

    # we want to add to NDVI_circoscrizioni the NDVI values for each circoscrizione. We want to save name of circoscrizione and the NDVI values (dt, data.median)
    NDVI_circoscrizioni[feature['properties']['nome']] = [(item['dt'], item['data']['median']) for item in data]

    # We have to delete the polygon because we have a limit on the area to use
    mgr.delete_polygon(the_new_polygon)
    
    print(feature['properties']['nome'])

APIRequestError: {"name":"PayloadTooLargeError","message":"You can not create polygons anymore."}

In [13]:
NDVI_circoscrizioni

{}

In [ ]:
""" mine_coordinates = [[77.3384, 28.6902], [74.5921, 28.7081], [72.0131, 24.6037], [86.5405, 25.0426], [82.7497, 22.3285]] #sample coordinates only
mine_area = ee.Geometry.MultiPoint(mine_coordinates).buffer(5000) #circular region of 5 kms radius around the point

def getNDVI(image):
    return image.normalizedDifference(['B8', 'B4'])

image_left = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2020-11-01', '2020-11-30').median().clip(mine_area)

ndvi_left = getNDVI(image_left) #input and output are images
ndviParams = {'min': 0,
               'max': 0.5,
               'palette': ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#ffffbf', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']}

mine_map = gm.Map(center = [24.922695, 77.531718], zoom = 6, lite_mode = True)
mine_map.addLayer(mine_area, {'color': 'red'})
mine_map.addLayer(ndvi_left, ndviParams, 'NDVI')
mine_map.addLayerControl()
mine_map

mineMeansFeatures = ndvi_left.reduceRegions(
    collection = mine_area,
    reducer = ee.Reducer.mean(),
    scale = 30,
)

ee.batch.Export.table.toDrive(collection = mineMeansFeatures, description = 'mineNDVI', fileFormat = 'CSV').start() """

In [1]:
import pandas as pd
import pysal as ps
import statsmodels.api as sm
import geopandas as gpd
import libpysal as lp
from spreg import ML_Lag
from spreg import OLS
import numpy as np
from sklearn.metrics import mean_squared_error as mse

# Load the data 
#data = pd.read_csv('data_trees_modified.csv')
data = gpd.read_file('../data/geojson/geo_data_trees.geojson')

# get the unique value of the Name column
data['Name'].unique()

/Users/luca/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/libpysal/cg/alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
/Users/luca/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/libpysal/cg/alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_faces(triangle):
/Users/luca/opt/anaconda3/e

array(['Common fig', 'Tamarisk spp', 'Plum spp', 'Black locust',
       'Common plum', 'English walnut', 'European black elderberry',
       'Japanese persimmon', 'Tree of heaven', 'European turkey oak',
       'European filbert', 'English oak', 'Elm spp', 'White mulberry',
       'Black poplar', 'Populus canescens', 'Ash spp', 'European ash',
       'White willow', 'Sweet cherry', 'European hackberry',
       'Flowering ash', 'Hedge maple', 'European hornbeam',
       'Narrow-leafed ash', 'Hawthorn spp', 'Blackthorn',
       'European medlar', 'Common pear', 'Apple spp', 'Black mulberry',
       'Japanese flowering cherry', 'Japanese privet',
       'Littleleaf linden', 'Basswood spp', 'Norway maple',
       'Necklacepod spp', 'Arbol de judea', 'Silver maple', 'Boxelder',
       'Silver linden', 'Norway spruce', 'Horse chestnut', 'Sycamore spp',
       'White poplar', 'English yew', 'Deodar cedar', 'Privet spp',
       'London planetree', 'Oriental planetree', 'Robinia spp',
       'A

In [2]:
len(data['Name'].unique())

237